# Pre-procesamiento de un Conjunto de Datos del Clima

En este notebook se muestra un ejemplo de preprocesamiento de datos, en este caso es un conjunto de datos climatico en Australia.

Este set de datos puede ser descargado del siguiente link:

https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

Se iniciara por cargar las bibliotecas necesarias para el pre-procesamiento

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
%matplotlib inline

Se comenzara por cargar los datos a memoria y mostrar una muestra de estos

In [ ]:
weather_results = pd.read_csv("weatherAUS.csv")
weather_results.sample(10)

Estos datos tienen varios problemas que deben de ser abordados antes de poder ser utilizados en un algoritmo.

* Se tienen valores de NaN que no dan informacion sobre el problema
* La direccion del viento viene especificada por una direccion en la brujula y no por un valor numerico
* Los valores de lluvia vienen especificados por valores de Si y No

Se compenzara por reemplazar `Yes` y `No` por valores numericos

In [ ]:
weather_results.replace(to_replace='Yes', value = 1, inplace = True)
weather_results.replace(to_replace='No',  value = 0, inplace = True)
weather_results.sample(10)

Los valores que aun no son numericos son los de direccion, los cuales se encuentran codificacods segun su direccion cardinal: `N`, `S`, etc..

Convertir estos valores a angulos no es practico dado que dos direcciones por ejemplo 1 y 359 sus valores incluso normalizados muy distintos a pesar de en la practica sus direcciones on muy iguales

Alternativas propuestas en https://stats.stackexchange.com/questions/218407/encoding-angle-data-for-neural-network y en https://stackoverflow.com/questions/20382484/periodic-data-with-machine-learning-like-degree-angles-179-is-2-different-fr son las de convertir este feature en dos features separados, representados por el coseno y seno de los angulos, este approach se usara para normalizar estos angulos.

In [ ]:
directions = ["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]

def compassToDeg(compass_direction):
    global directions
    index = directions.index(compass_direction)
    angle = index * 22.5
    return angle

def windGustDirConvert(direction):
    global directions
    if direction['WindGustDir'] in directions:
        return compassToDeg(direction['WindGustDir'])

def windDir9amConvert(direction):
    global directions
    if direction['WindDir9am'] in directions:
        return compassToDeg(direction['WindDir9am'])

def windDir3pmConvert(direction):
    global directions
    if direction['WindDir3pm'] in directions:
        return compassToDeg(direction['WindDir3pm'])

windGustDirAngle = weather_results.filter(regex=r'WindGustDir').apply(windGustDirConvert, axis=1)
windDir9amAngle = weather_results.filter(regex=r'WindDir9am').apply(windDir9amConvert, axis=1)
windDir3pmAngle = weather_results.filter(regex=r'WindDir3pm').apply(windDir3pmConvert, axis=1)

windGustDirAngleCos = windGustDirAngle.apply(np.cos)
windGustDirAngleSin = windGustDirAngle.apply(np.sin)
windDir9amAngleCos = windDir9amAngle.apply(np.cos)
windDir9amAngleSin = windDir9amAngle.apply(np.sin)
windDir3pmAngleCos = windDir3pmAngle.apply(np.cos)
windDir3pmAngleSin = windDir3pmAngle.apply(np.sin)

del weather_results['WindGustDir']
del weather_results['WindDir9am']
del weather_results['WindDir3pm']

weather_results['windGustDirAngleCos'] = windGustDirAngleCos
weather_results['windGustDirAngleSin'] = windGustDirAngleSin
weather_results['windDir9amAngleCos'] = windDir9amAngleCos
weather_results['windDir9amAngleSin'] = windDir9amAngleSin
weather_results['windDir3pmAngleCos'] = windDir3pmAngleCos
weather_results['windDir3pmAngleSin'] = windDir3pmAngleSin

weather_results.sample(10)

Ahora que todos los valores buscados son numericos se reemplazara `NaN` por la media respectiva

In [ ]:
weather_results.fillna(weather_results.mean(), inplace = True)
weather_results.sample(10)

A continuacion se muestra el codigo para normalizar todas las columnas, esto puede ser util, pero en este caso se obviara debido a que este no toma en cuenta la dependencia entre las distinas columnas.

Por ejemplo los valores de `MinTemp` y `MaxTemp`, los valores de velocidad del viento o de direccion de este deberian de ser normalizados en conjunto.

In [ ]:
# weather_numpy = weather_results.select_dtypes(include=[np.number])
# weather_norm = (weather_numpy - weather_numpy.min()) / (weather_numpy.max() - weather_numpy.min())
# weather_results[weather_norm.columns] = weather_norm

# weather_results.sample(10)

Finalmente el dato numerico restante corresponde a la ubicacion, este no se puede normalizar dado que estos no tienen un orden logico que sea representado por el mundo real.

Estos sin embargo pueden ser utilizados como entrada para la normalizacion de otras variables. Por ejemplo en lugar de usar la media, maximo y minimo global de todo el conjunto de datos estos se pueden normalizar segun la ubicacion, estoy puede evitar suprimir diferencias entre distintas ubicaciones como por ejemplo zonas costeras y montanhosas las cuales tendran distintos rangos.

Esto podria ser mezclado con la direccion del viento para 

# Visualizacion de datos

En la figura siguiente se puede observar como sus distribuciones tiene una distribucion gaussiana, como es de esperarse la temperatura maxima tiene una media mayor. En caso de que estos datos deseen normalizarse debe de mantenerse esta relacion.

In [ ]:
weather_results["MinTemp"].plot.hist(bins=20, alpha=0.5)
weather_results["MaxTemp"].plot.hist(bins=20, alpha=0.5)

Para otros parametros se tiene otro tipo de distribucion, por ejemplo en el caso de la lluvia la mayoria de los parametros indican una lluvia de 0. Todos los valores nulos tambien seran asignados a un valor cercano a este en una normalizacion.

In [ ]:
weather_results["Rainfall"].plot.hist(bins=20)

De igual manera en el caso del sol, dado que se utiliza la media para los valores desconocidos estos incrementan significativamente el valor al asignarse todos estos a la media. 

In [ ]:
weather_results["Sunshine"].plot.hist(bins=20)

In [ ]:
weather_results["WindGustSpeed"].plot.hist(bins=20, alpha=0.33)
weather_results["WindSpeed9am"].plot.hist(bins=20, alpha=0.33)
weather_results["WindSpeed3pm"].plot.hist(bins=20, alpha=0.33)

Los valores de direccion del viento tienen una distribucion uniforme lo que representa que el viento puede llegar de cualquier ubicacion.

In [ ]:
weather_results["windGustDirAngleCos"].plot.hist(bins=16, alpha=0.5)
weather_results["windGustDirAngleSin"].plot.hist(bins=16, alpha=0.5)

In [ ]:
weather_results["windDir9amAngleCos"].plot.hist(bins=16, alpha=0.5)
weather_results["windDir9amAngleSin"].plot.hist(bins=16, alpha=0.5)

In [ ]:
weather_results["windDir3pmAngleCos"].plot.hist(bins=16, alpha=0.5)
weather_results["windDir3pmAngleSin"].plot.hist(bins=16, alpha=0.5)